In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
import scipy.stats
from IPython.display import Video, display, HTML
sns.set()

def read_csvx(path, *args, **kwargs):
    try:
        return pd.read_csv(str(path) +'.gz', *args, **kwargs)
    except Exception as e:
        return pd.read_csv(str(path), *args, **kwargs)

def followorbit(vals, path):
    out = []
    for n in range(100):
        closest05 = np.clip(round(path[n] * 2) / 2, 0, 10)
        rank = vals.query(f'nbopp == {n} and othercoop == {closest05}')['rank'].values[0]
        out.append(rank)
    return np.array(out)

import re
def gethighest(vals):
        m = np.argmax([int(re.search('(\d+)', val).group(1)) for val in vals])
        return vals[m]

In [ ]:
dofolloworbit = True
for path in sorted(glob.glob("/home/pecoffet/remoterobo/logs/lions-nocost-2019-05-15-1759/lion/**/rep00/", recursive=True)):
    display(HTML(f'<h3>{path}</h3>'))
    fit = read_csvx(path + "fitnesslog.txt", delimiter="\t")
    fit = fit.groupby(['gen', 'ind'], as_index=False).sum()
    plt.figure()
    sns.lineplot(y='fitness', x='gen', data=fit, ci=None, estimator='median')
    sns.lineplot(y='fitness', x='gen', data=fit, ci=None, estimator='max')
    #sns.lineplot(y='fitness', x='gen', data=fit, ci=None, estimator='min')
    plt.show()
    logall = read_csvx(gethighest(glob.glob(path + 'logall*')), delimiter="\t")
    logall['oppId'] = logall['oppId'].astype('category')
    a = logall.query('id == 0 and eval == 0').pivot_table('id', 'iter', 'oppId', aggfunc='count', fill_value=0)
    for oppval in logall['oppId'].unique():
        if not a[oppval]:
            a[oppval] = 0
    # end of trick 
    plt.figure(figsize=(5, 20))
    sns.heatmap(a)
    plt.show()

FrozenList(['iter'])